In [ ]:
import numpy as np
import networkx as nx

In [ ]:
from src.visuals import visualize_graph, animate_trajectories, get_node_color_for_agent
from src.agent import ExploringAgent, TreeNode
from src.communication import find_groups, synchronize_information, update_tasks, no_collision_move
from src.graph_clearing import split_largest_subtree_until_threshold

In [ ]:
def generate_random_tree(n_nodes=10):
    end = np.arange(n_nodes)
    start = (end * np.random.uniform(size=n_nodes)).astype(int)
    edges = np.stack([start, end], axis=1)[1:]
    
    children = [edges[edges[:, 0] == i, 1].tolist() for i in range(n_nodes)]
    
    # Create nodes
    nodes = {i: TreeNode(i) for i in range(n_nodes)}
    
    # Build tree structure
    for i, child_indices in enumerate(children):
        for child_idx in child_indices:
            nodes[i].add_child(nodes[child_idx])
    
    G = nx.DiGraph()
    for parent, local_children in enumerate(children):
        for child in local_children:
            G.add_edge(parent, child)
            
    positions = nx.nx_agraph.graphviz_layout(G, prog='dot')
    positions = np.asarray([value for key, value in sorted(positions.items())])

    return nodes, edges, positions, children

In [ ]:
n_nodes = 10
true_map, edges, positions, children = generate_random_tree(n_nodes=n_nodes)

In [ ]:
n_agents = 2
n_responsibilities = 3
root = 0
agents = [ExploringAgent(i, root, true_map, root=root, all_responsibilities=list(range(n_responsibilities)), taken_responsibilities=list(range(n_agents))) for i in range(n_agents)]
trajectories = [[0] for _ in range(n_agents)]
explored_nodes = [[[]] for _ in range(n_agents)]

for i in range(150):
    connected_agent_groups = find_groups(agents, edges)
    for group in connected_agent_groups:
        synchronize_information(group)
        update_tasks(group, root)
        no_collision_move(group)
        
    for agent in agents:
        agent.update_map()
        trajectories[agent.id].append(agent.current_node)
        
    for idx, agent in enumerate(agents):
        explored_nodes[idx].append([k for k, v in agent.node_map.items() if v.get('visited', False)])

In [ ]:
fig = visualize_graph(n_nodes, positions, children)
animate_trajectories(fig, positions, trajectories, node_colors=get_node_color_for_agent(1, n_nodes, explored_nodes), highlight=1)

fig.show()

In [ ]:
split_largest_subtree_until_threshold(edges, 7)